In [1]:
import neptune

import json

import NIH.hemmer_functions as nhf

In [2]:
import os
import random
from itertools import chain
from typing import Any, Tuple
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data.dataset import Dataset
import torchvision
from torchvision import transforms
from PIL import Image
from sklearn.metrics import accuracy_score
from sklearn.model_selection import StratifiedKFold
from tqdm import tqdm
from tabulate import tabulate

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [3]:
with open('config.json', 'r') as f:
    config = json.load(f)

In [4]:
config_neptune = config["neptune"]
run = neptune.init_run(
    project=config_neptune["project"],
    api_token=config_neptune["api_token"],
)

<ipython-input-4-6ea371968b71>:2: NeptuneWarning: To avoid unintended consumption of logging hours during interactive sessions, the following monitoring options are disabled unless set to 'True' when initializing the run: 'capture_stdout', 'capture_stderr', and 'capture_hardware_metrics'.
  run = neptune.init_run(


https://app.neptune.ai/jonasl/masterarbeit/e/MAS-20


In [5]:
#params = config["params"]

params = {
    "NUM_CLASSES": 2,
    "DROPOUT": 0.00,
    "NUM_HIDDEN_UNITS": 30,
    "LR": 5e-3,
    "USE_LR_SCHEDULER": False,
    "TRAIN_BATCH_SIZE": 64,
    "TEST_BATCH_SIZE": 64,
    "EPOCHS": 20,
    "NUM_EXPERTS": 2,
    "K": 10,
    "TARGET": "Airspace_Opacity",
    "LABELER_IDS": [4323195249, 4295232296]
}

#choose radiologist_pair (last three digits are the IDS used in the paper)
#LABELER_IDS = [4295342357, 4295349121]
#LABELER_IDS = [4323195249, 4295194124]
#LABELER_IDS = [4295342357, 4295354117]

run["parameters"] = params

print(params)

{'NUM_CLASSES': 2, 'DROPOUT': 0.0, 'NUM_HIDDEN_UNITS': 30, 'LR': 0.005, 'USE_LR_SCHEDULER': False, 'TRAIN_BATCH_SIZE': 64, 'TEST_BATCH_SIZE': 64, 'EPOCHS': 2, 'NUM_EXPERTS': 2, 'K': 10, 'TARGET': 'Airspace_Opacity', 'LABELER_IDS': [4323195249, 4295232296]}


<ipython-input-5-b8d14e2a40ec>:23: NeptuneUnsupportedType: You're attempting to log a type that is not directly supported by Neptune (<class 'list'>).
        Convert the value to a supported type, such as a string or float, or use stringify_unsupported(obj)
        for dictionaries or collections that contain unsupported values.
        For more, see https://docs.neptune.ai/help/value_of_unsupported_type
  run["parameters"] = params


In [6]:
PATH = "../Datasets/NIH/"
PATH_Labels = PATH
PATH_Images = PATH

In [7]:
run["Expert Model"] = "old"

In [8]:
class NihExpert:
    """A class used to represent an Expert on NIH ChestX-ray data.

    Parameters
    ----------
    labeler_id : int
        the Reader ID to specify which radiologist the expert object represents
    target : str
        the target to make predictions for

    Attributes
    ----------
    labeler_id : int
        the Reader ID to specify which radiologist the expert object represents
    target : str
        the target to make predictions for
    image_id_to_prediction : dict of {int : str}
        a dictionary that maps the image id to the prediction the radiologist made for the specified target

    Methods
    -------
    predict(image_ids)
        makes a prediction for the given image ids
    """

    def __init__(self, labeler_id: int, target: str, PATH, numLabels=800, prob=0.5):
        self.labelerId = labeler_id
        self.target = target
        self.maxLabels = numLabels
        self.prob = prob
        
        self.resetPredictionCount()

        individual_labels = pd.read_csv(PATH + "labels.csv")

        expert_labels = individual_labels[individual_labels["Reader ID"] == self.labelerId][
            ["Image ID", self.target + "_Expert_Label", self.target + "_GT_Label"]]
        expert_labels = expert_labels.fillna(0)

        self.image_id_to_prediction = pd.Series(expert_labels[self.target + "_Expert_Label"].values,
                                                index=expert_labels["Image ID"]).to_dict()

    def predict(self, image_ids):
        """Returns the experts predictions for the given image ids. Works only for image ids that are labeled by the expert

        Parameters
        ----------
        image_ids : list of int
            the image ids to get the radiologists predictions for

        Returns
        -------
        list of int
            returns a list of 0 or 1 that represent the radiologists prediction for the specified target
        """
        return [self.image_id_to_prediction[image_id] for image_id in image_ids]

    def predict_unlabeled_data(self, image_ids):
        """Returns the experts predictions for the given image ids. Works for all image ids, returns -1 if not labeled by expert

        Parameters
        ----------
        image_ids : list of int
            the image ids to get the radiologists predictions for

        Returns
        -------
        list of int
            returns a list of 0 or 1 that represent the radiologists prediction for the specified target, or -1 if no prediction
        """
        return [self.image_id_to_prediction[image_id] if image_id in self.image_id_to_prediction else -1 for image_id in image_ids]
    
    def predictNew(self, image_ids):
        """
        Returns the expert prediction for the first n predictions
        For every other prediction is predicts with the probability (random guessing)
        """
        length = len(image_ids)
        if (self.predictions + length) <= self.maxLabels:
            self.predictions += length
            return [self.image_id_to_prediction[image_id] for image_id in image_ids]
        else:
            temp_predictions = [self.image_id_to_prediction[image_id] for image_id in image_ids[:(self.maxLabels - self.predictions)]]
            self.predictions = self.maxLabels
            for image_id in image_ids[(self.maxLabels - self.predictions):]:
                if np.random.uniform(0,1) > self.prob:
                    temp_predictions.append(self.image_id_to_prediction[image_id])
                else:
                    temp_predictions.append(np.random.randint(2, size=1))
    
    def resetPredictionCount(self):
        self.predictions = 0

class NihAverageExpert:
    def __init__(self, expert_fns=[]):
        self.expert_fns = expert_fns
        self.num_experts = len(self.expert_fns)

    def predict(self, filenames):
        all_experts_predictions = [expert_fn(filenames) for expert_fn in self.expert_fns]
        predictions = [None] * len(filenames)

        for idx, expert_predictions in enumerate(all_experts_predictions):
            predictions[idx::self.num_experts] = expert_predictions[idx::self.num_experts]

        return predictions

In [ ]:
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)

In [9]:
metrics = []
metrics_print = []

maxLabels = [4, 8, 16, 32, 128]

basic_Dataset = BasicDataset(PATH, "Airspace_Opacity")

for maxL in maxLabels:

    best_expert_accuracies = []
    avg_expert_accuracies = []
    our_approach_accuracies = []
    our_approach_coverages = []
    jsf_accuracies = []
    jsf_coverages = []
    full_automation_accuracies = []
    moae_accuracies = []
    mohe_accuracies = []

    for seed in range(3):
        print(f'Seed: {seed}')
        print("-"*40)
        #np.random.seed(seed)
        #random.seed(seed)
        
        set_seed(seed)

        nih_dataloader = ds.NIH_K_Fold_Dataloader(
        param["K"],
        param["LABELER_IDS"],
        param["TARGET"],
        param["TRAIN_BATCH_SIZE"],
        param["TEST_BATCH_SIZE"],
        seed,
        maxLabels=maxLabels,
        PATH=PATH
        )
    
        expert_fns = []
        for labelerId in list(params["LABELER_IDS"]):
            nih_expert = NihExpert(labeler_id=labelerId, target=params["TARGET"], PATH=PATH)
            expert_fns.append(nih_expert.predict)

        best_expert_accuracy = nhf.get_accuracy_of_best_expert(seed, nih_dataloader, expert_fns, PATH, maxL, param=params)
        best_expert_accuracies.append(best_expert_accuracy)
        run[f"maxLabels_{maxL}/seed_{seed}/best_expert_accuracy"].append(best_expert_accuracy)
    
        avg_expert_accuracy = nhf.get_accuracy_of_average_expert(seed, nih_dataloader, expert_fns, PATH, maxL, param=params)
        avg_expert_accuracies.append(avg_expert_accuracy)
        run[f"maxLabels_{maxL}/seed_{seed}/avg_expert_accuracy"].append(avg_expert_accuracy)

        our_approach_accuracy, our_approach_coverage = nhf.run_team_performance_optimization("Our Approach", seed, nih_dataloader, expert_fns, PATH, maxL, param=params)
        our_approach_accuracies.append(our_approach_accuracy)
        our_approach_coverages.append(our_approach_coverage)
        run[f"maxLabels_{maxL}/seed_{seed}/our_approach_accuracy"].append(our_approach_accuracy)
    
        jsf_accuracy, jsf_coverage = nhf.run_team_performance_optimization("Joint Sparse Framework", seed, nih_dataloader, expert_fns, PATH, maxL, param=params)
        jsf_accuracies.append(jsf_accuracy)
        jsf_coverages.append(jsf_coverage)
        run[f"maxLabels_{maxL}/seed_{seed}/jsf_accuracy"].append(jsf_accuracy)

        full_automation_accuracy = nhf.run_full_automation(seed, nih_dataloader, PATH, maxL, param=params)
        full_automation_accuracies.append(full_automation_accuracy)
        run[f"maxLabels_{maxL}/seed_{seed}/full_automation_accuracy"].append(full_automation_accuracy)

        moae_accuracy = nhf.run_moae(seed, nih_dataloader, PATH, maxL, param=params)
        moae_accuracies.append(moae_accuracy)
        run[f"maxLabels_{maxL}/seed_{seed}/moae_accuracy"].append(moae_accuracy)

        mohe_accuracy = nhf.run_mohe(seed, nih_dataloader, expert_fns, PATH, maxL, param=params)
        mohe_accuracies.append(mohe_accuracy)
        #run[f"maxLabels_{maxL}/seed_{seed}/mohe_accuracy"].append(mohe_accuracy)
        print("-"*40)
        
    mean_best_expert_accuracy = np.mean(best_expert_accuracies)
    mean_best_expert_coverage = 0.00

    mean_avg_expert_accuracy = np.mean(avg_expert_accuracies)
    mean_avg_expert_coverage = 0.00

    mean_our_approach_accuracy = np.mean(our_approach_accuracies)
    mean_our_approach_coverage = np.mean(our_approach_coverages)

    mean_jsf_accuracy = np.mean(jsf_accuracies)
    mean_jsf_coverage = np.mean(jsf_coverages)

    mean_full_automation_accuracy = np.mean(full_automation_accuracies)
    mean_full_automation_coverage = 100.00

    mean_moae_accuracy = np.mean(moae_accuracies)
    mean_moae_coverage = 100.00

    mean_mohe_accuracy = np.mean(mohe_accuracies)
    mean_mohe_coverage = 0.00
    
    metrics_print.append(mean_best_expert_accuracy)
    metrics_print.append(mean_best_expert_coverage)
    run[f"maxLabels_{maxL}/mean_best_expert_accuracy"].append(mean_best_expert_accuracy)
    
    metrics_print.append(mean_avg_expert_accuracy)
    metrics_print.append(mean_avg_expert_coverage)
    run[f"maxLabels_{maxL}/mean_avg_expert_accuracy"].append(mean_avg_expert_accuracy)
    
    metrics_print.append(mean_our_approach_accuracy)
    metrics_print.append(mean_our_approach_coverage)
    run[f"maxLabels_{maxL}/mean_our_approach_accuracy"].append(mean_our_approach_accuracy)
    
    metrics_print.append(mean_jsf_accuracy)
    metrics_print.append(mean_jsf_coverage)
    run[f"maxLabels_{maxL}/mean_jsf_accuracy"].append(mean_jsf_accuracy)
    
    metrics_print.append(mean_full_automation_accuracy)
    metrics_print.append(mean_full_automation_coverage)
    run[f"maxLabels_{maxL}/mean_full_automation_accuracy"].append(mean_full_automation_accuracy)
    
    metrics_print.append(mean_moae_accuracy)
    metrics_print.append(mean_moae_coverage)
    run[f"maxLabels_{maxL}/mean_moae_accuracy"].append(mean_moae_accuracy)
    
    metrics_print.append(mean_mohe_accuracy)
    metrics_print.append(mean_mohe_coverage)
    run[f"maxLabels_{maxL}/mean_mohe_accuracy"].append(mean_mohe_accuracy)
    
    
    metrics.append(best_expert_accuracies)
    metrics.append(avg_expert_accuracies)
    metrics.append(our_approach_accuracies)
    metrics.append(our_approach_coverages)
    metrics.append(jsf_accuracies)
    metrics.append(jsf_coverages)
    metrics.append(full_automation_accuracies)
    metrics.append(moae_accuracies)
    metrics.append(mohe_accuracies)

Seed: 0
----------------------------------------
Running fold 1 out of 10


/home/joli/joli-env/lib/python3.9/site-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=10.
  warnings.warn(


Running fold 2 out of 10
Running fold 3 out of 10
Running fold 4 out of 10
Running fold 5 out of 10
Running fold 6 out of 10
Running fold 7 out of 10
Running fold 8 out of 10
Running fold 9 out of 10
Running fold 10 out of 10
Best Expert Accuracy: 0.9131455399061033

Running fold 1 out of 10


/home/joli/joli-env/lib/python3.9/site-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=10.
  warnings.warn(


Running fold 2 out of 10
Running fold 3 out of 10
Running fold 4 out of 10
Running fold 5 out of 10
Running fold 6 out of 10
Running fold 7 out of 10
Running fold 8 out of 10
Running fold 9 out of 10
Running fold 10 out of 10
Average Expert Accuracy: 0.8509389671361502

Team Performance Optimization with Our Approach
load Resnet-18 pretrained on ImageNet


...

/home/joli/joli-env/lib/python3.9/site-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=10.
  warnings.warn(


Running fold 2 out of 10
Running fold 3 out of 10
Running fold 4 out of 10
Running fold 5 out of 10
Running fold 6 out of 10
Running fold 7 out of 10
Running fold 8 out of 10
Running fold 9 out of 10
Running fold 10 out of 10
Average Expert Accuracy: 0.8626760563380281

Team Performance Optimization with Our Approach
load Resnet-18 pretrained on ImageNet


/home/joli/joli-env/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/joli/joli-env/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
/home/joli/joli-env/lib/python3.9/site-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=10.
  warnings.warn(
  0%|          | 0/2 [00:00<?, ?it/s]

Running fold 1 out of 10


TypeError: 'NoneType' object is not subscriptable

In [14]:
met_save = metrics

In [31]:
len(metrics_print)

56

In [39]:
met = []
for j in [0, 2, 4, 6, 8, 10, 12]:
    temp = []
    for i in [0, 14, 28, 42]:
        temp.append(metrics_print[i + j])
    met.append(temp)
met
for elm in met[0]:
    run["metrics/mean_best_expert_accuracy"].append(elm)
for elm in met[1]:
    run["metrics/mean_avg_expert_accuracy"].append(elm)
for elm in met[2]:
    run["metrics/mean_our_approach_accuracy"].append(elm)
for elm in met[3]:
    run["metrics/mean_jsf_accuracy"].append(elm)
for elm in met[4]:
    run["metrics/mean_full_automation_accuracy"].append(elm)
for elm in met[5]:
    run["metrics/mean_moae_accuracy"].append(elm)
for elm in met[6]:
    run["metrics/mean_mohe_accuracy"].append(elm)

In [ ]:
run[f"maxLabels_{maxL}/mean_jsf_accuracy"].append(mean_jsf_accuracy)
run[]

for maxLabel in range(5):
    for seed in range(3):
        run[f"metrics"

In [29]:
import itertools
ks = [i + j for i, j in itertools.product([0, 9, 18, 27], [0, 1, 2, 4, 6, 7, 8])]
met = []
for i in ks:
    met.append(metrics[i])
met

[[0.9131455399061033, 0.9131455399061033, 0.9131455399061033],
 [0.8509389671361502, 0.8591549295774648, 0.8474178403755869],
 [0.744131455399061, 0.7922535211267606, 0.7664319248826291],
 [0.7382629107981221, 0.687793427230047, 0.6784037558685446],
 [0.539906103286385, 0.4847417840375587, 0.49178403755868544],
 [0.49295774647887325, 0.4894366197183099, 0.4835680751173709],
 [0.8392018779342723, 0.8532863849765259, 0.8509389671361502],
 [0.9131455399061033, 0.9131455399061033, 0.9131455399061033],
 [0.8591549295774648, 0.8356807511737089, 0.8685446009389671],
 [0.7922535211267606, 0.7464788732394366, 0.7382629107981221],
 [0.6948356807511737, 0.8227699530516432, 0.7147887323943662],
 [0.505868544600939, 0.5223004694835681, 0.49178403755868544],
 [0.5657276995305164, 0.4964788732394366, 0.5011737089201878],
 [0.8321596244131455, 0.8415492957746479, 0.8485915492957746],
 [0.9131455399061033, 0.9131455399061033, 0.9131455399061033],
 [0.8615023474178404, 0.8403755868544601, 0.859154929577

In [18]:
metrics

[[0.9131455399061033, 0.9131455399061033, 0.9131455399061033],
 [0.8509389671361502, 0.8591549295774648, 0.8474178403755869],
 [0.744131455399061, 0.7922535211267606, 0.7664319248826291],
 [305, 156, 221],
 [0.7382629107981221, 0.687793427230047, 0.6784037558685446],
 [323, 447, 421],
 [0.539906103286385, 0.4847417840375587, 0.49178403755868544],
 [0.49295774647887325, 0.4894366197183099, 0.4835680751173709],
 [0.8392018779342723, 0.8532863849765259, 0.8509389671361502],
 [0.9131455399061033, 0.9131455399061033, 0.9131455399061033],
 [0.8591549295774648, 0.8356807511737089, 0.8685446009389671],
 [0.7922535211267606, 0.7464788732394366, 0.7382629107981221],
 [201, 197, 266],
 [0.6948356807511737, 0.8227699530516432, 0.7147887323943662],
 [274, 161, 395],
 [0.505868544600939, 0.5223004694835681, 0.49178403755868544],
 [0.5657276995305164, 0.4964788732394366, 0.5011737089201878],
 [0.8321596244131455, 0.8415492957746479, 0.8485915492957746],
 [0.9131455399061033, 0.9131455399061033, 0.913

In [13]:
for met in metrics:
    
    mean_best_expert_accuracy = met[0]
    mean_best_expert_coverage = 0.00

    mean_avg_expert_accuracy = met[2]
    mean_avg_expert_coverage = 0.00

    mean_our_approach_accuracy = met[4]
    mean_our_approach_coverage = met[5]

    mean_jsf_accuracy = met[6]
    mean_jsf_coverage = met[7]

    mean_full_automation_accuracy = met[8]
    mean_full_automation_coverage = 100.00

    mean_moae_accuracy = met[10]
    mean_moae_coverage = 100.00

    mean_mohe_accuracy = met[12]
    mean_mohe_coverage = 0.00
    
    print(tabulate([['Our Approach', mean_our_approach_accuracy, mean_our_approach_coverage],
                ['JSF', mean_jsf_accuracy, mean_jsf_coverage],
                ['--------', '--------', '--------'],
                ['Full Automation', mean_full_automation_accuracy, mean_full_automation_coverage],
                ['MOAE', mean_moae_accuracy, mean_moae_coverage],
                ['MOHE', mean_mohe_accuracy, mean_mohe_coverage],
                ['Random Expert', mean_avg_expert_accuracy, mean_avg_expert_coverage],
                ['Best Expert', mean_best_expert_accuracy, mean_best_expert_coverage]], 
               headers=['Method', 'Accuracy', 'Coverage']))

IndexError: list index out of range

In [ ]:
run.stop()

In [8]:
class NihExpert:
    """A class used to represent an Expert on NIH ChestX-ray data.

    Parameters
    ----------
    labeler_id : int
        the Reader ID to specify which radiologist the expert object represents
    target : str
        the target to make predictions for

    Attributes
    ----------
    labeler_id : int
        the Reader ID to specify which radiologist the expert object represents
    target : str
        the target to make predictions for
    image_id_to_prediction : dict of {int : str}
        a dictionary that maps the image id to the prediction the radiologist made for the specified target

    Methods
    -------
    predict(image_ids)
        makes a prediction for the given image ids
    """

    def __init__(self, labeler_id: int, target: str, PATH, numLabels=800, prob=0.5):
        self.labelerId = labeler_id
        self.target = target
        self.maxLabels = numLabels
        self.prob = prob
        
        self.resetPredictionCount()

        individual_labels = pd.read_csv(PATH + "labels.csv")

        expert_labels = individual_labels[individual_labels["Reader ID"] == self.labelerId][
            ["Image ID", self.target + "_Expert_Label", self.target + "_GT_Label"]]
        expert_labels = expert_labels.fillna(0)

        self.image_id_to_prediction = pd.Series(expert_labels[self.target + "_Expert_Label"].values,
                                                index=expert_labels["Image ID"]).to_dict()

    def predict(self, image_ids):
        """Returns the experts predictions for the given image ids. Works only for image ids that are labeled by the expert

        Parameters
        ----------
        image_ids : list of int
            the image ids to get the radiologists predictions for

        Returns
        -------
        list of int
            returns a list of 0 or 1 that represent the radiologists prediction for the specified target
        """
        return [self.image_id_to_prediction[image_id] for image_id in image_ids]

    def predict_unlabeled_data(self, image_ids):
        """Returns the experts predictions for the given image ids. Works for all image ids, returns -1 if not labeled by expert

        Parameters
        ----------
        image_ids : list of int
            the image ids to get the radiologists predictions for

        Returns
        -------
        list of int
            returns a list of 0 or 1 that represent the radiologists prediction for the specified target, or -1 if no prediction
        """
        return [self.image_id_to_prediction[image_id] if image_id in self.image_id_to_prediction else -1 for image_id in image_ids]
    
    def predictNew(self, image_ids):
        """
        Returns the expert prediction for the first n predictions
        For every other prediction is predicts with the probability (random guessing)
        """
        length = len(image_ids)
        if (self.predictions + length) <= self.maxLabels:
            self.predictions += length
            return [self.image_id_to_prediction[image_id] for image_id in image_ids]
        else:
            temp_predictions = [self.image_id_to_prediction[image_id] for image_id in image_ids[:(self.maxLabels - self.predictions)]]
            self.predictions = self.maxLabels
            for image_id in image_ids[(self.maxLabels - self.predictions):]:
                if np.random.uniform(0,1) > self.prob:
                    temp_predictions.append(self.image_id_to_prediction[image_id])
                else:
                    temp_predictions.append(np.random.randint(2, size=1))
    
    def resetPredictionCount(self):
        self.predictions = 0

class NihAverageExpert:
    def __init__(self, expert_fns=[]):
        self.expert_fns = expert_fns
        self.num_experts = len(self.expert_fns)

    def predict(self, filenames):
        all_experts_predictions = [expert_fn(filenames) for expert_fn in self.expert_fns]
        predictions = [None] * len(filenames)

        for idx, expert_predictions in enumerate(all_experts_predictions):
            predictions[idx::self.num_experts] = expert_predictions[idx::self.num_experts]

        return predictions